# Режим шифрования Propagating Cipher Block Chaining (РСВС)

Недостатки режима CBC привели к созданию усовершенстванного режима распространяющегося сцепления блоков шифра (Propagating Cipher Block Chaining, РСВС). Естественно, этот режим похож на CBC за исключением того, что предыдущий блок открытого текста и предыдущий блок шифртекста подвергается операции XOR с текущим блоком открытого текста перед шифрованием или после него

Шифрование:

$$ {\displaystyle c_{i}=E_{k}\left(m_{i}\oplus m_{i-1}\oplus c_{i-1}\right)}$$

Дешифрование:

$${\displaystyle m_{i}=D_{k}(c_{i})\oplus c_{i-1}\oplus m_{i-1}}m_{i}$$

Схема шифрования:



### Получение входных данных


In [16]:
import math

print("Введите ключ:")
key = input()

print("Введите вектор инициаллизациии:")
vector = input()

print("Введите фразу для шифрования:")
phrase = input()

# разбивание фразы на блоки по 16 символов
blocks = []
for i in range(0, math.ceil(len(phrase)/16)):
    blocks.append(phrase[i*16 : (i+1)*16 if (i+1)*16 < len(phrase) else len(phrase)])
    
print(f"Блоки для шифрования: {blocks}")  

gen_key(key, phrase)
encode(phrase)

Введите ключ:
it is key
Введите вектор инициаллизациии:
this is my words
Введите фразу для шифрования:
this is my words
Блоки для шифрования: ['this is my words']


IndexError: list index out of range

# Функция шифрования

Подробно, с коммернариями и выводом в файле rc6.ipynb

In [15]:
def ROR(x, n, bits = 32):
    mask = (2**n) - 1
    mask_bits = x & mask
    return (x >> n) | (mask_bits << (bits - n))

#rotate left input x, by n bits
def ROL(x, n, bits = 32):
    return ROR(x, bits - n,bits)

# операция округления до ближайшего нечетного целого
def odd(x):
    if x - math.trunc(x) == 0:
        return x + 1
    return math.ceil(x) if math.ceil(x) % 2 == 1 else math.floor(x)

# константа золотого сечения
phi = (1 + 5 ** 0.5) / 2

w = 32
r = 12
b = len(key)
K = []
W = []

# генерация констант
p_w = odd( (math.e - 2) * math.pow(2, w) )
q_w = odd( (phi - 1) * math.pow(2, w) ) 

S = []
L = []

u = w/8
t = 2 * r + 4

def gen_key(phrase, key):

    # Приведение ключа к необходимому размеру
    for i in range(len(key), 16):
        key += " "
    key = key[:16]

    # Приведение фразы к необходимому размеру
    for i in range(len(phrase), 16):
        phrase += " "
    phrase = phrase[:16]

    # Ключ
    for i in range(0, b):
        K.append(ord(key[i]))

    # фраза
    for i in range(0, len(phrase)):
        W.append(ord(phrase[i]))
    
    c = math.ceil(max(b, 1) / u)

    for i in range(0, c):
        L.append(0)
        for j in range(0, math.trunc(b/c)):
            L[i] = ROL(L[i], 8) + K[math.trunc(i * u + j)]  
            
    S.append(p_w)

    for i in range(1, t):
        S.append((S[i-1] + q_w) % (2**w))
        
        A = B = C = D = i = j = 0

    l = L[::-1]
    s = S[:]

    v = 3 * max(c, t)

    for q in range(0, v):
        A = s[i] = ROL((s[i] + A + B) % 2**32 , 3)
        B = l[j] = ROL((l[j] + A + B) % 2**32, (A + B) % 32)
        i = (i + 1) % t
        j = (j + 1) % c
        
    print(L)
    print(S)
        
def encode(phrase):
    L1 = []
    l = L[::-1]
    s = S[:]
    cur = 0
    for i in range(0, 4):
        L1.append(0)
        for j in range(0, math.trunc(b/4)):
            L1[i] = ROL(L1[i], 8) + W[cur]
            cur += 1
    
    A, B, C, D = L1[0], L1[1], L1[2], L1[3]

    B = (B + s[0]) % 2**32
    D = (D + s[1]) % 2**32

    def shafl(X1, X2, X3, X4):
        return X1, X2, X3, X4

    for i in range(1, r + 1):
        print([A, B, C, D])
        temp = ROL((B * (2 * B + 1)) % 2**32, int(math.log(w, 2)))
        u_temp = ROL((D * (2 * D + 1)) % 2**32, int(math.log(w, 2)))
        A = ROL( (A ^ temp) % 2**32, u_temp % 32) + s[2 * i]
        C = ROL( (C ^ u_temp) % 2**32 , temp % 32) + s[2*i + 1]
        A, B, C, D = shafl(B, C, D, A) #  (A, B, C, D) = (B, C, D, A)
                                       #  -- параллельное присвоения значений.
    A = (A + s[2*r + 2]) % 2**32
    C = (C + s[2*r + 3]) % 2**32

    encode = [A, B, C, D]
    
    res = ""
    for e in encode:
        temp =bin(e)[2:]
        if len(temp) <32:
            temp = "0"*(32-len(temp)) + temp
        for i in range(0,4):
            res=res+chr(int(temp[i*8:(i+1)*8],2))

    print(f'Готовая зашифроаванная фраза: "{res}"')
    
    return res

def decode(phrase):
    
    A, B, C, D = encode[0], encode[1], encode[2], encode[3]

    C = (C - s[2*r + 3]) % 2**32
    A = (A - s[2*r + 2]) % 2**32

    for i in reversed(range(1, r + 1)):
        A, B, C, D = shafl(D, A, B, C)   #  (A, B, C, D) = (D, A, B, C) 
                                         #  -- параллельное присвоения значений.
        u_temp = ROL((D * (2*D + 1)) % 2**32, int(math.log(w, 2)))
        temp = ROL((B * (2*B + 1)) % 2**32, int(math.log(w, 2)))
        C = (ROR((C - s[2*i + 1]) % 2**32, temp % 32) ^ u_temp)
        A = (ROR((A - s[2*i]) % 2**32, u_temp % 32) ^ temp)

    D = (D - s[1]) % 2**32
    B = (B - s[0]) % 2**32

    decode = [A, B, C, D]
    print(f'A = {A}, \nB = {B}, \nC = {C}, \nD = {D}')
    
    res1 = ""
    for e in decode:
        temp =bin(e)[2:]
        if len(temp) <32:
            temp = "0"*(32-len(temp)) + temp
        for i in range(0,4):
            res1=res1+chr(int(temp[i*8:(i+1)*8],2))

    print(f'Готовая расшифроаванная фраза: "{res1}"')
    return res